In [ ]:
import torch 
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms

from torch.nn.modules.loss import CrossEntropyLoss

# Introduction

https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/01-basics/pytorch_basics/main.py

In [ ]:
# Create tensors.
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

In [ ]:
# Build a computational graph.
y = w * x + b    # y = 2 * x + 3

In [ ]:
# Compute gradients.
y.backward()

In [ ]:
# Print out the gradients.
# Calculated using partial derivatives
print("x.grad = {}".format(x.grad.item()))  # x.grad = 2 
print("w.grad = {}".format(w.grad.item()))  # w.grad = 1 
print("b.grad = {}".format(b.grad.item()))  # b.grad = 1

In [ ]:
# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)

In [ ]:
# Build a fully connected layer.
linear = nn.Linear(3, 2)
print ('w: ', linear.weight)
print ('b: ', linear.bias)

In [ ]:
# Build loss function and optimizer.
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(linear.parameters(), lr=1e-4)

In [ ]:
# Forward pass.
pred = linear(x)

In [ ]:
# Compute loss.
loss = criterion(pred, y)
print('loss: ', loss.item())

In [ ]:
# Print out the gradients.
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

In [ ]:
# 1-step gradient descent.
optimizer.step()

In [ ]:
# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

# Example

Train Resnet18 model on the CIFAR10 dataset.

In [ ]:
# Download and construct CIFAR10 dataset.

# Mean and std for Resnet18 model pretrained on ImageNet
means = [0.485, 0.456, 0.406]
stds = [0.229, 0.224, 0.225]

t = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=means,
                         std=stds)]
)

# Load dataset and apply transforms
cifar10_dataset = torchvision.datasets.CIFAR10(root='./data/',
                                           train=True, 
                                           transform=t,
                                           download=True)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

# https://jamesmccaffrey.wordpress.com/2020/08/07/displaying-cifar-10-images-using-pytorch/
def imshow(img, means, stds):
    original_img = torch.empty(img.shape)
    for idx in range(3):
        original_img[idx] = (img[idx] * stds[idx]) + means[idx]
        
    npimg = original_img.numpy()   # convert from tensor
    plt.imshow(np.transpose(npimg, (1, 2, 0))) 
    plt.show()

In [ ]:
# Labels for CIFAR10

cifar10_idx_to_label = {
    0: "airplane",
    1: "automobile",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck"
}

cifar10_idx_to_label

In [ ]:
# Example 1
imshow(cifar10_dataset[50][0], means, stds), cifar10_idx_to_label[cifar10_dataset[50][1]]

In [ ]:
# Example 2
imshow(cifar10_dataset[100][0], means, stds), cifar10_idx_to_label[cifar10_dataset[100][1]]

In [ ]:
# Split data into training and validation datasets (80% & 20%)
train_set, val_set = torch.utils.data.random_split(cifar10_dataset, [int(0.8*len(cifar10_dataset)), 
                                                                     int(0.2*len(cifar10_dataset))])
# Fetch one data pair (read data from disk).
image, label = train_set[0]
print (image.size())
print (label)

In [ ]:
# Data loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_set,
                                           batch_size=64, 
                                           shuffle=True)

val_loader = torch.utils.data.DataLoader(dataset=val_set,
                                           batch_size=64, 
                                           shuffle=True)

In [ ]:
# Download and load the ResNet-18.
# Pretrained on ImageNet, but dataset too big for this tutorial.
resnet = torchvision.models.resnet18(pretrained=True)

# Only finetune the last layer.
for param in resnet.parameters():
    param.requires_grad = False

# Replace the top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 10)

In [ ]:
# Build loss function and optimizer.
criterion = CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet.parameters(), lr=1e-3, weight_decay=1e-6)

In [ ]:
# Training

# Save accuracy and losses
accs, losses = [], []

n_epochs = 100
for epoch in range(n_epochs):
    
    for idx, batch in enumerate(train_loader):

        resnet.train()
        
        # Calculate loss
        pred, y = batch[0], batch[1]
        output = resnet(pred)
        loss = criterion(output, y)
        
        # Backpropagate loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Calculate predicted label and accuracy
        yhat = torch.argmax(output.detach(), dim=1)
        acc = torch.sum(torch.eq(yhat, y).view(-1)) / yhat.shape[0]
        
        accs.append(acc.item())
        losses.append(loss.item())

        print("Epoch: {}, Batch Index: {}\nAccuracy: {}\nLoss: {}\n\n".format(epoch,
                                                                          idx,
                                                                          acc.item(), 
                                                                          loss.item()))

In [ ]:
# Plot accuracy and loss over time
fig, axs = plt.subplots(1, 2, figsize=(20,10))

axs[0].plot([i for i in range(len(accs))], accs, c="orange")
axs[0].set_title("Accuracy vs. Time")
axs[0].set_xlabel("Iterations")
axs[0].set_ylabel("Accuracy")

axs[1].plot([i for i in range(len(losses))], losses)
axs[1].set_title("Loss vs. Time")
axs[1].set_xlabel("Iterations")
axs[1].set_ylabel("Loss")

In [ ]:
# Exercise: validate the model!